In [ ]:
import warnings
import pandas as pd
import numpy as np

from statsmodels.tsa.seasonal import seasonal_decompose

# imports de modulos que estao definidos dentro do projeto
import data
import arima
import timeseries

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
timeseries02_daily = data.load_timeseries02_daily()

timeseries02_daily.plot(figsize=(15, 10), linewidth=2, fontsize=15, style='b.')
plt.grid(True)
plt.xlabel('Ano', fontsize=15)

In [ ]:
timeseries02_monthly = data.load_timeseries02_monthly()

timeseries02_monthly.plot(figsize=(15, 10), linewidth=2, fontsize=15, style='r.')
plt.grid(True)
plt.xlabel('Ano', fontsize=15)

In [ ]:
timeseries.check_stationarity(timeseries02_monthly, window=12)

In [ ]:
# Apply a nonlinear log transformation
# Log Scale Transformation
timeseries02_monthly_log = np.log(timeseries02_monthly)
timeseries.check_stationarity(timeseries02_monthly_log, window=12)

In [ ]:
result = seasonal_decompose(timeseries02_monthly_log, model='additive')
#result = seasonal_decompose(timeseries02_monthly_log, model='multiplicative')
result.plot()
plt.show()

In [ ]:
movingAverage = timeseries02_monthly_log.rolling(window=12).mean()
t1 = timeseries02_monthly_log - movingAverage
t1.dropna(inplace=True)
timeseries.check_stationarity(t1, window=12)

In [ ]:
# Remove trend and seasonality with differencing
# Time Shift Transformation 
ts_euro_week_log_diff = timeseries02_monthly_log - timeseries02_monthly_log.shift()
plt.plot(ts_euro_week_log_diff)

In [ ]:
ts_euro_week_log_diff.dropna(inplace=True)
timeseries.check_stationarity(ts_euro_week_log_diff, window=12)

In [ ]:
#Exponential Decay Transformation
exponentialDecayWeightedAverage = timeseries02_monthly_log.ewm(halflife=12, min_periods=0, adjust=True).mean()
datasetLogScaleMinusExponentialMovingAverage = timeseries02_monthly_log - exponentialDecayWeightedAverage
timeseries.check_stationarity(datasetLogScaleMinusExponentialMovingAverage, window=12)

In [ ]:
decomposition = seasonal_decompose(timeseries02_monthly_log) 

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

decomposedLogData = residual
decomposedLogData.dropna(inplace=True)
timeseries.check_stationarity(decomposedLogData, window=12)

In [ ]:
warnings.filterwarnings("ignore")

arima_params = arima.get_arima_params(
    p_values=range(0, 3),
    d_values=range(0, 2),
    q_values=range(0, 2))
#print(len(arima_params))

#arima.evaluate_models(timeseries02_monthly.values, arima_params)
arima.evaluate_models(decomposedLogData.values, arima_params)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

#X = timeseries02_monthly.values
X = decomposedLogData.values

size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()


model = ARIMA(history, order=(2, 1, 1))
model_fit = model.fit(disp=0)
#output = model_fit.forecast()
forecast = model_fit.forecast(steps=len(test))[0]
#print(forecast)
#print(type(forecast))

for yhat in forecast:
    predictions.append(yhat)
    obs = test[t]
    #print('predicted=%f, expected=%f' % (yhat, obs))

print(predictions)
#print(len(predictions))
#print(len(test))

error = mean_squared_error(test, predictions)
print('Test MSE: %.9f' % error)

plt.figure(figsize=(20, 15))
plt.plot(test, color='blue')
plt.plot(predictions, color='red')
plt.show()